In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

# #low old binning (wrong?)
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

#low new binning
fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

# #high neew binning
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")

k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]); # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
type(check)
check._members();

In [ ]:
def smoothgauss(MFPAD, sigmax, sigmay):
    """
    Fucntion the smooths with a 2D gaussian function 2D matrixes. For the experiments
    x = cos(theta), y = phi. Phi is twice ctheta and sigma should be the same.
    """
    # NOTE: the right order is y,x (from stackoverflow)
    sigma = [sigmay,sigmax] 
    Y = sp.ndimage.filters.gaussian_filter(MFPAD, sigma, mode="constant")
    return Y

def sorting_array(inarray, cosphi, a):
    """
    a is the level according to which values have to be sorted: 1 = cos_light, 2 = phi_light
    Function to sort the input array according to the labels attached from LMF2ROOT. The lables are stored in cosphi during the loaing.
    To load sorting the 72 items according to cos(theta) use level = 1, according to phi level = 2. The first level is a tring, thereofere
    is not possible to sort according to it.
    NOTE: for experimental data the default order is according to phi, civecersa for the theoretical MFPADs.
    """
    cosn=np.array([col[0] for col in cosphi]); #list
    phin=np.array([col[1] for col in cosphi]); #list
    if inarray.ndim>2:
        data = inarray.reshape(72, inarray.shape[1]*inarray.shape[2]).T
        df = pd.DataFrame(
            data=data,
            #NOTE: phiM and cosM should be passed into the function
            index=pd.MultiIndex.from_product([phiM, cosM],names=["phi","cos(theta)"]),
            columns=['item {}'.format(i) for i in range(72)])
            
    else:
        cosbin = inarray.shape[1]        
        data = inarray.T
        df = pd.DataFrame(
            data=data,
            index=np.linspace(-1,1,inarray.shape[1]),
            columns=['item {}'.format(i) for i in range(72)])
        
    df1=df.T
    df1["cos_light"]=cosn
    df1["phi_light"]=phin
    df1.set_index("cos_light", append=True, inplace=True) #level=1
    df1.set_index("phi_light", append=True, inplace=True) #level=2
    
    if inarray.ndim>2:
        dfnew=df1
    else:
        #here it transforms the DataBAse into a series in order to sort correctly the cos(theta) column too
        dfnew=df1.stack()
    
    #level=2 is the standard for experiment, level=1 for theory
    #kind="mergesort" the concept of stable sorting algorithm
    dfnew.sort_index(level=a, inplace=True, kind="mergesort")
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes cos_light
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes phi
    
    if inarray.ndim>2:
        #fast approach with numpy with transpose to rearrange the order of the axes:
        outarray=(dfnew.T).values.reshape(inarray.shape[1],inarray.shape[2],72).transpose(2, 0, 1)
    else:
        outarray=dfnew.values.reshape(72, inarray.shape[1])
        
    return outarray

def cosphi_func(key, cosphi):
    ctheta_nc = float((str(key).split("costheta_")[1]).split("_phi")[0])
    phi_nc = float((str(key).split("phi_")[1]).split(";")[0])
    # alternative method with import re
    #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1)
    cosphi.append((ctheta_nc, phi_nc))
    return cosphi

def normalise_matrix(a):
    row_sums = a.sum(axis=1)
    new_matrix = a / row_sums[:, np.newaxis]
    return new_matrix

def shift_func(a, flag=0.):
    if a.shape[0]>12:
        c=a.reshape(12,6)
        flag=1.
        if c[0][0] == c[0][1]:
            b=(c[1][0]-c[0][0])/2
            a=np.add(c,b)
        else:
            b=(c[0][1]-c[0][0])/2
            a=np.add(c,b)
    else:
        if a[0][0] == a[0][1]:
            b=(a[1][0]-a[0][0])/2
            a=np.add(a,b)
        else:
            b=(a[0][1]-a[0][0])/2
            a=np.add(a,b)
    if flag == 1.:
        # a=np.ravel(a)
        a=a.reshape(-1)
    return a

def import_MFPAD(file, MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred, run_MFPAD=0., run_cos=0.):
    """
    Loads the MFPADs and the cos(theta) from the .root files.
    6 inputs + a parameter
    NOTE: MFPAD_xy and ctheta_c have originally +1 dimensions compare to the z values.
    For the sake of iminiut, cos(theta) is centered on the middle of the bins.
    """
    for key, value in file[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            cosphi=cosphi_func(key,cosphi)
            temp=np.array(file[filename].numpy())
            MFPAD.append(temp[0]) # it is a list!
            if run_MFPAD == 0.:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_MFPAD=1.
        elif "cos(theta)" in filename.lower():
            temp=np.array(file[filename].numpy())
            ctheta.append(temp[0]) # it is a list!
            if run_cos == 0.:
                ctheta_c.append(temp[1])
                ctheta_cred.append(np.array((ctheta_c[0][1:] + ctheta_c[0][:-1])/2)) #! reduced of 1 dimension
                run_cos=1.
    return MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred

In [ ]:
# initilialises a list
MFPAD_RCR=[]; ctheta_RCR=[]; ctheta_xaxis_RCR=[]; cosphi_RCR=[];
MFPAD_RCL=[]; ctheta_RCL=[]; ctheta_xaxis_RCL=[]; cosphi_RCL=[];
MFPAD_SCR=[]; ctheta_SCR=[]; ctheta_xaxis_SCR=[]; cosphi_SCR=[];
MFPAD_SCL=[]; ctheta_SCL=[]; ctheta_xaxis_SCL=[]; cosphi_SCL=[];

MFPAD_xy=[];
ctheta_cred=[];

import_MFPAD(fileRCR, MFPAD_RCR, cosphi_RCR, MFPAD_xy, ctheta_RCR, ctheta_xaxis_RCR, ctheta_cred)
import_MFPAD(fileRCL, MFPAD_RCL, cosphi_RCL, MFPAD_xy, ctheta_RCL, ctheta_xaxis_RCL, ctheta_cred)
import_MFPAD(fileSCR, MFPAD_SCR, cosphi_SCR, MFPAD_xy, ctheta_SCR, ctheta_xaxis_SCR, ctheta_cred)
import_MFPAD(fileSCL, MFPAD_SCL, cosphi_SCL, MFPAD_xy, ctheta_SCL, ctheta_xaxis_SCL, ctheta_cred)

# converting into nparray
MFPAD_RCR=np.array(MFPAD_RCR); ctheta_RCR=np.array(ctheta_RCR); ctheta_xaxis_RCR=np.array(ctheta_xaxis_RCR[0]); cosphi_RCR=np.array(cosphi_RCR);
MFPAD_RCL=np.array(MFPAD_RCL); ctheta_RCL=np.array(ctheta_RCL); ctheta_xaxis_RCL=np.array(ctheta_xaxis_RCL[0]); cosphi_RCL=np.array(cosphi_RCL);
MFPAD_SCR=np.array(MFPAD_SCR); ctheta_SCR=np.array(ctheta_SCR); ctheta_xaxis_SCR=np.array(ctheta_xaxis_SCR[0]); cosphi_SCR=np.array(cosphi_SCR);
MFPAD_SCL=np.array(MFPAD_SCL); ctheta_SCL=np.array(ctheta_SCL); ctheta_xaxis_SCL=np.array(ctheta_xaxis_SCL[0]); cosphi_SCL=np.array(cosphi_SCL);

ctheta_cred=ctheta_cred[0];
cosphi=cosphi_RCR;

In [ ]:
print(MFPAD_RCR.shape)
print(MFPAD_RCR.max())
print(MFPAD_RCR.min())

In [ ]:
cosM=(MFPAD_xy[0][1][1:] + MFPAD_xy[0][1][:-1])/2

In [ ]:
phiM=(MFPAD_xy[0][0][1:] + MFPAD_xy[0][0][:-1])/2

In [ ]:
plt.contourf(phiM,cosM,MFPAD_RCR[0].T)
plt.legend(loc="best")
MFPAD_RCL[0].T.shape

In [ ]:
cosphi.shape

In [ ]:
MFPAD_RCR.shape

In [ ]:
MFPAD_RCR_cos=sorting_array(MFPAD_RCR, cosphi, 1)
MFPAD_RCR_phi=sorting_array(MFPAD_RCR, cosphi, 2)

MFPAD_RCL_cos=sorting_array(MFPAD_RCL, cosphi, 1)
MFPAD_SCR_cos=sorting_array(MFPAD_SCR, cosphi, 1)
MFPAD_SCL_cos=sorting_array(MFPAD_SCL, cosphi, 1)

#interesting but slow approach with xarray
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import xarray as xr
MFPAD_RCR_xarray = dMFPAD_RCR_cos.to_xarray().to_array();

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_xarray[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

#----------------------------------------------------------------------------#

fig = go.Figure()

fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCR_xarray[j+i*6].T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=24., size=6)), row=j+1, col=i+1)
    
fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=True,
    width=1000,
    # height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_cos[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

#just a test: as to be identical to the unsorted one

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR_phi[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

#unsorted version

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCR[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCL_cos[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCR_cos[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
# fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
# for i,ax in zip(range(72),axes.T.flatten()) :
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_SCL_cos[i].T)
    ax.text(0, 0, cosphi[i].T, ha="center", va="center", color="red",fontsize=18)
    
fig.tight_layout()
plt.show()

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6!

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows=6, cols=12, vertical_spacing=0.015)

for i in range(12):
    for j in range(6):
        fig.add_trace(go.Contour(x=phiM,y=cosM,z=MFPAD_RCR_cos[j+i*6].T, line_smoothing=0.7, legendgroup="no", colorscale="Viridis", showscale=False, showlegend=False, contours=dict(start=0., end=24., size=6)), row=j+1, col=i+1)
    
fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=True,
    width=1000,
    # height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()